# ELMo-LSA-SVM Model
- Uses GPU if available
- Creates pickle files for the ELMo embeddings under "/embeddings/ELMo/"

In [1]:
## turn off warnings for cleaner execution
import os, shutil
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

## IMPORTS
# GLOVE Embeddings
import spacy

# ELMo
import tensorflow as tf
import tensorflow_hub as hub
import pickle
import time
import os
from sklearn.model_selection import train_test_split

# SVM
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, ParameterGrid, StratifiedKFold
import scipy.stats as stats
from joblib import parallel_backend

# Preprocessing
import csv
import chardet
import numpy as np
import re
import pandas as pd
import demoji
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
import json

# metrics
from sklearn.model_selection import StratifiedKFold, learning_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, balanced_accuracy_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

2024-11-30 02:41:41.598477: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732905701.611272    8205 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732905701.614617    8205 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/walnuts/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# check if gpu is being used
num_gpu = len(tf.config.list_physical_devices('GPU'))
print("Num GPUs Available: ", num_gpu)

print("Using GPU" if num_gpu > 0 else "Not Using GPU")

Num GPUs Available:  1
Using GPU


In [3]:
# Load the dataset / read csv
# MAKE SURE CSV IS IN UTF-8 (if tweets have emojis)

## Testing if csv can open (Use for troubleshooting)
# with open('Eng_Tweets.csv') as csv_file:
#   csvFile = csv.reader(csv_file, delimiter=',')
#   for row in csv_file:
#     print(row)

## RUN PARAMETERS
LANG = 1  # 1 = English | 2 = Tagalog | 3 = TagLish | 4 = Mixed
DO_GRIDSEARCH = False
GENERATE_NEW_ELMO = True
COMBO_METHOD = 1 # 1 = Concatenation | 2 = Averaging
manual_params = {
    'probability': True,
    'C': 1,
    'kernel': 'linear',
    'class_weight': None,
    'random_state': 42
}

pd.set_option('display.max_colwidth', None)

# Assign the CSV path based on LANG value
if LANG == 1:
    csv_path = 'Eng_Tweets.csv'
elif LANG == 2:
    csv_path = 'Tag_Tweets.csv'
elif LANG == 3:
    csv_path = 'TagLish_Tweets.csv'
elif LANG == 4:
    csv_path = 'Mixed_Tweets.csv'
else:
    raise ValueError("Invalid value for LANG. Must be 1, 2, 3, or 4.")


df = pd.read_csv(csv_path)
print("total rows: " + str(len(df)))
df.head() ## head won't show emojis unless using print function

total rows: 2338


,Username,Tweet,Date posted,Label
0,@fudaishii,i'm genuinely going to attempt tonight i can't do this anymore i can't handle all this stress i wish i was never born bro,9/11/24,1
1,@yourdystiny,"Becoming less reactive is a huge part of growth & decreasing stress. If you let everything get you worked up, you’ll damage your mind, body & spirit.",9/8/24,0
2,@ocenhxu,me ??? tired ??? stressed ??? exhausted ??? i wanna cry ??? yes.,9/8/24,1
3,@ifeelgoodto,"skipping meals, irregular sleeping habits, overthinking, stress, tired and drained. that's me, that's my everyday life",9/10/24,1
4,@ysuckme,"you deserve to be happy. not confused, not hurt, not stressed, just happy.",9/8/24,0


## Preprocessing
- Remove mentions (@)
- Remove hashtags (#)
- Remove URLs
- Replace emojis with textual description (Using demoji)
- Remove stop words

In [4]:
### PREPROCESSING STEPS (FOR TAGALOG)
with open('tl_stopwords.json', 'r') as f:
    tagalog_stop_words = json.load(f)

# Preprocessing functions
def clean_text_tl(text):
    # Remove mentions
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    # Replace emoji with textual descriptions
    text = demoji.replace_with_desc(text)
    text = re.sub(r'(:[a-zA-Z\s]+:)', r' \1 ', text)  # Add spaces around the shortcode
    text = re.sub(r'(:[a-zA-Z\s]+:)', lambda match: match.group(0).replace(' ', '_'), text)
    text = text.strip()
    return text

def remove_stopwords_tl(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in tagalog_stop_words]
    return ' '.join(filtered_words)

In [5]:
### PREPROCESSING STEPS
def clean_text(text):
  # Remove mentions
  text = re.sub(r'@[A-Za-z0-9_]+', '', text)
  # Remove hashtags
  text = re.sub(r'#\w+', '', text)
  # Remove URLs
  text = re.sub(r'http\S+|www\S+|https\S+', '', text)
  # Replace emoji with textual descriptions
  text = demoji.replace_with_desc(text)
  text = re.sub(r'(:[a-zA-Z\s]+:)', r' \1 ', text) # Add spaces around the shortcode
  text = re.sub(r'(:[a-zA-Z\s]+:)', lambda match: match.group(0).replace(' ', '_'), text)
  text = text.strip()
  return text

stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
  words = text.split()
  filtered_words = [word for word in words if word.lower() not in stop_words]
  return ' '.join(filtered_words)

In [6]:
## Execute preprocessing
df = df.dropna()  # Get rid of NaN rows

if LANG == 1:
    print("EXECUTING ENGLISH TWEET CLEANING")
    df['Tweet'] = df['Tweet'].apply(clean_text)
    df['Tweet'] = df['Tweet'].apply(remove_stopwords)
elif LANG == 2:
    print("EXECUTING TAGALOG TWEET CLEANING")
    df['Tweet'] = df['Tweet'].apply(clean_text_tl)
    df['Tweet'] = df['Tweet'].apply(remove_stopwords_tl)
else:
    print("EXECUTING BOTH ENGLISH AND TAGALOG TWEET CLEANING")
    df['Tweet'] = df['Tweet'].apply(clean_text_tl)
    df['Tweet'] = df['Tweet'].apply(remove_stopwords_tl)
    df['Tweet'] = df['Tweet'].apply(clean_text)
    df['Tweet'] = df['Tweet'].apply(remove_stopwords)
    

df.head()

EXECUTING ENGLISH TWEET CLEANING


,Username,Tweet,Date posted,Label
0,@fudaishii,i'm genuinely going attempt tonight can't anymore can't handle stress wish never born bro,9/11/24,1
1,@yourdystiny,"Becoming less reactive huge part growth & decreasing stress. let everything get worked up, you’ll damage mind, body & spirit.",9/8/24,0
2,@ocenhxu,??? tired ??? stressed ??? exhausted ??? wanna cry ??? yes.,9/8/24,1
3,@ifeelgoodto,"skipping meals, irregular sleeping habits, overthinking, stress, tired drained. that's me, that's everyday life",9/10/24,1
4,@ysuckme,"deserve happy. confused, hurt, stressed, happy.",9/8/24,0


In [7]:
df['Label'].value_counts()

Label
0    1897
1     441
Name: count, dtype: int64

## GloVe Embeddings
- Make sure to run "python -m spacy download en_core_web_md"
- GloVe embeddings are then averaged and standardized
- Train-Test-Validation split = 70% - 20% - 10%
- Uses undersampling to balance the training set

In [8]:
nlp = spacy.load("en_core_web_md", disable=["tagger", "parser", "ner"]) # disable non word-embedding parts of spacy

# Create an empty list to store the embeddings
embeddings = []

# Iterate through the DataFrame
for index, row in df.iterrows():
    tweet = row['Tweet']
    doc = nlp(tweet)

    # Get the embeddings for each token and store them
    tweet_embeddings = [token.vector for token in doc]
    embeddings.append(tweet_embeddings)

# Add the embeddings to the DataFrame as a new column
df['embeddings'] = embeddings

/home/walnuts/projects/tf-gpu/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/walnuts/projects/tf-gpu/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/walnuts/projects/tf-gpu/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/walnuts/projects/tf-gpu/lib/python3.12/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check

In [9]:
# Function to average embeddings for a tweet
def average_embeddings(embeddings_list):
    if embeddings_list:
        return np.mean(embeddings_list, axis=0)
    else:
        return np.zeros(300)  # 300 = embedding dimension

In [10]:
# Function to split the dataset into train, validation, and test
def split_data(df, train_size=0.7, val_size=0.1, test_size=0.2):
    # 70% into training 30% into validation + testing 
    train_df, temp_df = train_test_split(df, train_size=train_size, random_state=42)
    
    # 30% splits 10% into validation and 20% into testing
    val_df, test_df = train_test_split(temp_df, train_size=val_size / (val_size + test_size), random_state=42)
    
    return train_df, val_df, test_df

In [11]:
# Average the sentence embedding into a single entry
df['avg_embedding'] = df['embeddings'].apply(average_embeddings)

# Split the dataset
train_df, val_df, test_df = split_data(df)

## Run ELMo in batches
- Splits data into train-test-validate (70%-20%-10%)
- Uses the default ELMo model
- Max Sequence Length for embedding padding
- Done in batches to avoid computational overload

In [12]:
# Batch processing elmo with padding
def process_in_batches(df, batch_size=50, max_seq_length=280, pickle_dir="embeddings/ELMo/"):
    num_rows = len(df)
    
    # Ensure the directory exists for saving pickled files
    os.makedirs(pickle_dir, exist_ok=True)
    
    # Iterate over the dataframe in chunks of batch_size
    for start_idx in range(0, num_rows, batch_size):
        end_idx = min(start_idx + batch_size, num_rows)
        batch = df["Tweet"].iloc[start_idx:end_idx]

        # Measure time for each batch
        start = time.time()
        embeddings_tensor = elmo(tf.constant(batch))["elmo"]
        end = time.time()
        
        length = end - start
        print(f"Processed rows {start_idx} to {end_idx - 1}")
        print("Time Taken for batch: ", length, "seconds")
        print("Embedding Shape for batch: ", embeddings_tensor.shape, "\n")
        
        # Pad all sequences in the batch to the fixed max length
        padded_batch_embeddings = []
        for i in range(embeddings_tensor.shape[0]):
            seq_length = embeddings_tensor[i].shape[0]
            padding_needed = max_seq_length - seq_length
            # Pad the sequence with zeros (pad to match the max length)
            padded_seq = tf.pad(embeddings_tensor[i], [[0, padding_needed], [0, 0]], mode='CONSTANT')
            padded_batch_embeddings.append(padded_seq)
        
        # Stack the padded embeddings into a tensor
        padded_batch_embeddings = tf.stack(padded_batch_embeddings)

        # Save the embeddings for this batch to a pickle file
        pickle_file = os.path.join(pickle_dir, f"batch_{start_idx}_embeddings.pkl")
        with open(pickle_file, 'wb') as f:
            pickle.dump(padded_batch_embeddings.numpy(), f)
        
        print(f"Saved embeddings for batch {start_idx} to {pickle_file}")

In [13]:
# Checkk flag for generating new elmo embeddings
def check_and_process_data(df, batch_size, max_seq_length, pickle_dir):
    if not GENERATE_NEW_ELMO:
        print(f"Skipping processing as GENERATE_NEW_ELMO is set to False.")
        return
        
    else:
        # If the directory exists, delete it
        if os.path.exists(pickle_dir):
            print(f"Directory {pickle_dir} exists. Deleting it to generate new embeddings.")
            shutil.rmtree(pickle_dir)
            
        print(f"Processing data for {pickle_dir}...")
        start_total = time.time()
        process_in_batches(df, batch_size=batch_size, max_seq_length=max_seq_length, pickle_dir=pickle_dir)
        end_total = time.time()
        print(f"Total Time for {pickle_dir}: {end_total - start_total} seconds")

In [14]:
# Load ELMo default model
print("Downloading elmo model at: \"https://tfhub.dev/google/elmo/2\"")
elmo = hub.load("https://tfhub.dev/google/elmo/2").signatures["default"]
print("Loaded elmo model")

# Sequence length for padding (should be >= max tweet length in dataset)
MAX_SEQ_LEN = 140

# Assuming 'df' is the dataframe with your dataset and the column containing text is "Tweet"
# train_df, val_df, test_df = split_data(df, train_size=0.7, val_size=0.1, test_size=0.2)

# Directory paths for embeddings
train_emb_dir = "embeddings/ELMo/train/"
val_emb_dir = "embeddings/ELMo/val/"
test_emb_dir = "embeddings/ELMo/test/"

# Process the train, validation, and test data
print("Processing train dataset...")
start_total = time.time()
# process_in_batches(train_df, batch_size=50, max_seq_length=MAX_SEQ_LEN, pickle_dir="embeddings/ELMo/train/")
check_and_process_data(train_df, batch_size=50, max_seq_length=MAX_SEQ_LEN, pickle_dir=train_emb_dir)
end_total = time.time()
print(f"Total Time for Training Set: {end_total - start_total} seconds")

print("Processing validation dataset...")
start_total = time.time()
# process_in_batches(val_df, batch_size=50, max_seq_length=MAX_SEQ_LEN, pickle_dir="embeddings/ELMo/val/")
check_and_process_data(val_df, batch_size=50, max_seq_length=MAX_SEQ_LEN, pickle_dir=val_emb_dir)
end_total = time.time()
print(f"Total Time for Validation Set: {end_total - start_total} seconds")

print("Processing test dataset...")
start_total = time.time()
# process_in_batches(test_df, batch_size=50, max_seq_length=MAX_SEQ_LEN, pickle_dir="embeddings/ELMo/test/")
check_and_process_data(test_df, batch_size=50, max_seq_length=MAX_SEQ_LEN, pickle_dir=test_emb_dir)
end_total = time.time()
print(f"Total Time for Test Set: {end_total - start_total} seconds")

total_time = end_total - start_total
print("Generated Embeddings for entire dataset")
print("Total Time Taken: ", total_time, "seconds")

I0000 00:00:1732905712.616344    8205 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3539 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Loaded elmo model
Processing train dataset...
Directory embeddings/ELMo/train/ exists. Deleting it to generate new embeddings.
Processing data for embeddings/ELMo/train/...


I0000 00:00:1732905719.997467    8274 cuda_dnn.cc:529] Loaded cuDNN version 90300


Processed rows 0 to 49
Time Taken for batch:  8.801424741744995 seconds
Embedding Shape for batch:  (50, 51, 1024) 

Saved embeddings for batch 0 to embeddings/ELMo/train/batch_0_embeddings.pkl
Processed rows 50 to 99
Time Taken for batch:  1.0367248058319092 seconds
Embedding Shape for batch:  (50, 33, 1024) 

Saved embeddings for batch 50 to embeddings/ELMo/train/batch_50_embeddings.pkl
Processed rows 100 to 149
Time Taken for batch:  2.12335467338562 seconds
Embedding Shape for batch:  (50, 108, 1024) 

Saved embeddings for batch 100 to embeddings/ELMo/train/batch_100_embeddings.pkl
Processed rows 150 to 199
Time Taken for batch:  0.8827493190765381 seconds
Embedding Shape for batch:  (50, 29, 1024) 

Saved embeddings for batch 150 to embeddings/ELMo/train/batch_150_embeddings.pkl
Processed rows 200 to 249
Time Taken for batch:  1.3080182075500488 seconds
Embedding Shape for batch:  (50, 55, 1024) 

Saved embeddings for batch 200 to embeddings/ELMo/train/batch_200_embeddings.pkl
Pro

KeyboardInterrupt: 

## Combining GloVe and ELMo Embeddings
- Load ELMo Embeddings
- Combine it with the GloVe Embeddings via vstack, average pooling, or Principal Component Analysis
- Feed this combined embedding into the SVM

In [ ]:
# def load_elmo_embeddings(pickle_dir, num_batches):
def load_elmo_embeddings(pickle_dir, batch_start=0, batch_step=50, max_batches=None):
    elmo_embeddings = []
    batch_index = batch_start
    batch_count = 0
    
    while True:
        pickle_file = f"{pickle_dir}/batch_{batch_index}_embeddings.pkl"
        
        try:
            # Open the pickle file
            with open(pickle_file, 'rb') as f:
                batch_embeddings = pickle.load(f)
                # Average across the sequence length to get a fixed-size embedding
                batch_avg_embeddings = np.mean(batch_embeddings, axis=1)
                elmo_embeddings.extend(batch_avg_embeddings)
        except FileNotFoundError:
            print(f"File not found: {pickle_file}. Stopping batch loading.")
            break
        
        # Update batch index
        batch_index += batch_step
        batch_count += 1
        
        # Stop if the maximum number of batches is reached
        if max_batches is not None and batch_count >= max_batches:
            break
    
    return np.array(elmo_embeddings)

# Load ELMo embeddings for training data
train_elmo_embeddings = load_elmo_embeddings(
    pickle_dir="embeddings/ELMo/train",
    batch_start=0,    # Start from batch_0_embeddings.pkl
    batch_step=50,    # Increment by 50 for subsequent batches
    max_batches=None  # Load all available batches
)

# Load ELMo embeddings for validation data
val_elmo_embeddings = load_elmo_embeddings(
    pickle_dir="embeddings/ELMo/val",
    batch_start=0,
    batch_step=50
)

# Load ELMo embeddings for test data
test_elmo_embeddings = load_elmo_embeddings(
    pickle_dir="embeddings/ELMo/test",
    batch_start=0,
    batch_step=50
)


# Ensure the embeddings align with DataFrame rows
assert len(train_elmo_embeddings) == len(train_df)
assert len(val_elmo_embeddings) == len(val_df)
assert len(test_elmo_embeddings) == len(test_df)

In [ ]:
train_elmo_embeddings.shape

In [ ]:
train_elmo_embeddings

In [ ]:
# Function to pad the GloVe (300) embeddings to 1024 dimensions
def pad_glove_to_elmo(glove_embeddings, target_dim=1024):
    if glove_embeddings.shape[0] < target_dim:
        padding = np.zeros(target_dim - glove_embeddings.shape[0])
        return np.concatenate([glove_embeddings, padding])
    else:
        return glove_embeddings

# Pad GloVe embeddings to 1024 dimensions
def average_embeddings_spacy_elmo(spacy_embeddings, elmo_embeddings):
    # Pad Spacy embeddings to 1024 dimensions if necessary
    padded_spacy = pad_glove_to_elmo(spacy_embeddings)
    
    # Now average the padded Spacy embeddings and ELMo embeddings
    return (padded_spacy + elmo_embeddings) / 2


In [ ]:
# X_train = np.array(train_df['avg_embedding'].to_list())
y_train = train_df['Label']
# X_val = np.array(val_df['avg_embedding'].to_list())
y_val = val_df['Label']
# X_test = np.array(test_df['avg_embedding'].to_list())
y_test = test_df['Label']

# Concatenate SpaCy and ELMo embeddings
if COMBO_METHOD == 1:
    # Concatenation
    X_train = np.hstack((np.array(train_df['avg_embedding'].to_list()), train_elmo_embeddings))
    X_val = np.hstack((np.array(val_df['avg_embedding'].to_list()), val_elmo_embeddings))
    X_test = np.hstack((np.array(test_df['avg_embedding'].to_list()), test_elmo_embeddings))
else:
    # Averaging
    X_train = np.array([average_embeddings_spacy_elmo(spacy_emb, elmo_emb) 
                           for spacy_emb, elmo_emb in zip(train_df['avg_embedding'].to_list(), train_elmo_embeddings)])
    
    X_val = np.array([average_embeddings_spacy_elmo(spacy_emb, elmo_emb) 
                         for spacy_emb, elmo_emb in zip(val_df['avg_embedding'].to_list(), val_elmo_embeddings)])
    
    X_test = np.array([average_embeddings_spacy_elmo(spacy_emb, elmo_emb) 
                          for spacy_emb, elmo_emb in zip(test_df['avg_embedding'].to_list(), test_elmo_embeddings)])


# UNDERSAMPLING
# undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)
# X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

# OVERSAMPLING
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print(f"Class distribution after undersampling:\n{pd.Series(y_train_resampled).value_counts()}")

In [ ]:
print(f"Train set shape: {X_train_resampled.shape}")
print(f"Validation set shape: {X_val.shape}")
print(f"Test set shape: {X_test.shape}")

In [ ]:
X_val

## SVM (Support Vector Machine)

In [ ]:
# Modify param_grid for linear kernel
if DO_GRIDSEARCH:
    param_grid_linear = {
        'C': [0.1, 1, 10],
        'kernel': ['linear'],  # Only linear kernel
        'class_weight': [None, 'balanced'],  # Class weight
    }
    
    # Stratified K Fold
    stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # Initialize the SVC model
    svc = SVC(probability=True, random_state=42)
    
    # Define the GridSearchCV object with StratifiedKFold
    grid_search_linear = GridSearchCV(
        estimator=svc,
        param_grid=param_grid_linear,
        cv=stratified_kfold,
        n_jobs=-1,
        verbose=3,
        scoring='roc_auc'
    )
    
    # Fit the grid search to the training data
    with parallel_backend('multiprocessing'):
        grid_search_linear.fit(X_train_resampled, y_train_resampled)
    
    # Get the best parameters and best score for linear kernel
    best_params_linear = grid_search_linear.best_params_
    best_score_linear = grid_search_linear.best_score_
    
    # Print the results for linear kernel
    print(f"Best parameters for linear kernel: {best_params_linear}")
    print(f"Best cross-validation Accuracy for linear kernel: {best_score_linear:.4f}")
else:
    print("Skipped GridSearchCV")

In [ ]:
# Using the best parameters from grid search or random search
if DO_GRIDSEARCH:
    model = grid_search.best_estimator_  # or 
    # model = random_search.best_estimator_
else:
    model = SVC(**manual_params, verbose=True)
    model.fit(X_train_resampled, y_train_resampled)

# Final evaluation on the test set
print("RESULTS:")
y_pred_test = model.predict(X_test)
accuracy_test = accuracy_score(y_test, y_pred_test)
print(f"Test Accuracy: {accuracy_test}")

In [ ]:
# Evaluation on validation set
# y_pred_val = best_model.predict(X_val)
y_pred_val = model.predict(X_val)
accuracy_val = accuracy_score(y_val, y_pred_val)
precision_val = precision_score(y_val, y_pred_val)
recall_val = recall_score(y_val, y_pred_val)
f1_val = f1_score(y_val, y_pred_val)
balanced_acc_val = balanced_accuracy_score(y_val, y_pred_val)  # Add balanced accuracy
conf_matrix_val = confusion_matrix(y_val, y_pred_val)

print(f"Validation Accuracy: {accuracy_val}")
print(f"Validation Precision: {precision_val}")
print(f"Validation Recall: {recall_val}")
print(f"Validation F1 Score: {f1_val}")
print(f"Validation Balanced Accuracy: {balanced_acc_val}")  # Print balanced accuracy
print(f"Validation Confusion Matrix: \n{conf_matrix_val}")

# Final evaluation on test set
# y_pred_test = best_model.predict(X_test)
y_pred_test = model.predict(X_test)
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test)
recall_test = recall_score(y_test, y_pred_test)
f1_test = f1_score(y_test, y_pred_test)
balanced_acc_test = balanced_accuracy_score(y_test, y_pred_test)  # Add balanced accuracy
conf_matrix_test = confusion_matrix(y_test, y_pred_test)

print(f"Test Accuracy: {accuracy_test}")
print(f"Test Precision: {precision_test}")
print(f"Test Recall: {recall_test}")
print(f"Test F1 Score: {f1_test}")
print(f"Test Balanced Accuracy: {balanced_acc_test}")  # Print balanced accuracy
print(f"Test Confusion Matrix: \n{conf_matrix_test}")

# Compute ROC AUC
# y_prob_test = best_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class
y_prob_test = model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class
roc_auc = roc_auc_score(y_test, y_prob_test)
print(f"Test ROC AUC: {roc_auc}")

# Plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_prob_test)
plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Curve')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Plot confusion matrix for test set
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix_test, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Predicted Negative', 'Predicted Positive'], 
            yticklabels=['Actual Negative', 'Actual Positive'])
plt.title('Confusion Matrix - Test Set')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Plot confusion matrix for validation set
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix_val, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Predicted Negative', 'Predicted Positive'], 
            yticklabels=['Actual Negative', 'Actual Positive'])
plt.title('Confusion Matrix - Validation Set')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
# Using StratifiedKFold for cross-validation
stratified_cv = StratifiedKFold(n_splits=5)

train_sizes, train_scores, test_scores = learning_curve(
    model, X_train_resampled, y_train_resampled, cv=stratified_cv,  # Use StratifiedKFold
    scoring='accuracy', 
    train_sizes=np.linspace(0.1, 1.0, 10), 
    n_jobs=-1
)

# Calculate the mean and standard deviation of train and test scores
train_scores_mean = train_scores.mean(axis=1)
train_scores_std = train_scores.std(axis=1)
test_scores_mean = test_scores.mean(axis=1)
test_scores_std = test_scores.std(axis=1)

# Print the final scores (for the largest training set size)
print(f"Final Training Score: {train_scores_mean[-1]:.4f}")
print(f"Final Cross-validation Score: {test_scores_mean[-1]:.4f}")

# Plot the learning curve
plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_scores_mean, label='Training score', color='blue')
plt.plot(train_sizes, test_scores_mean, label='Cross-validation score', color='green')

# Fill the area between the curve and the axis to represent the standard deviation
plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, color='blue', alpha=0.2)
plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, color='green', alpha=0.2)

# Add labels and title
plt.xlabel('Training Set Size')
plt.ylabel('Accuracy')
plt.title('Learning Curve')
plt.legend(loc='best')
plt.grid(True)
plt.show()

In [ ]:
import os
import pickle

os.makedirs('saved_models/' + path, exist_ok=True)

# save the model
model_path = 'saved_models/' + path + '/Glove-Elmo-SVM(' + method + path + '.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(model, f)

print("Model saved successfully!")